# core

> Main codebase for fetching and saving RSS feeds

In [ ]:
# | default_exp core

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import json
import logging
from pathlib import Path
from typing import *

import fire
import reader
import rich
from bs4 import BeautifulSoup
from fastcore.basics import patch_to
from rich import print
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()

logger = logging.getLogger(__name__)


def strip_html(s: str):
    s = BeautifulSoup(s, "html.parser")
    return s.text

In [ ]:
# | export

class Feed:
    """RSS feed class"""

    def __init__(self, url: str, name: str = None, tags: List[str] = []):
        self.url = url
        self.name = name
        self.tags = tags

    def add_tag(self, tag: str):
        if tag not in self.tags:
            self.tags.append(tag)

    def remove_tag(self, tag: str):
        if tag in self.tags:
            self.tags.remove(tag)

In [ ]:
# | export


class PyNewsReader:
    def __init__(self, feeds=List[Feed]):
        self._dbfolder = Path().home() / ".cache/pynewsreader"
        if not self._dbfolder.exists():
            self._dbfolder.mkdir(parents=True)

        # If custom feed names exist, read them in
        feed_names_path = self._dbfolder / "feed_names.json"
        if feed_names_path.exists():
            with open(self._dbfolder / "feed_names.json", "rt") as myfile:
                self._feed_names = json.load(myfile)
        else:
            self._feed_names = {}

        # If title blacklist exists, read it in
        title_blacklist_path = self._dbfolder / "title_blacklist.json"
        if title_blacklist_path.exists():
            with open(self._dbfolder / "title_blacklist.json", "rt") as myfile:
                self._title_blacklist = json.load(myfile)
        else:
            self._title_blacklist = []

        self._reader = reader.make_reader(
            self._dbfolder / "db.sqlite",
            plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"],
        )

        # If title whitelist exists, read it in
        title_whitelist_path = self._dbfolder / "title_whitelist.json"
        if title_whitelist_path.exists():
            with open(self._dbfolder / "title_whitelist.json", "rt") as myfile:
                self._title_whitelist = json.load(myfile)
        else:
            self._title_whitelist = []

        self._reader = reader.make_reader(
            self._dbfolder / "db.sqlite",
            plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"],
        )

        self._reader.enable_search()

    def _print_entries(
        self, entries: List[reader.Entry], mark_as_read: bool = True, limit: int = 10
    ):
        """Pretty print entries - supports reader.Reader.get_entries arguments"""
        displayed_links = set()
        for e in entries:
            if e.link in displayed_links:
                # Don't display duplicates
                self._reader.mark_entry_as_read(e)
            else:
                displayed_links.add(e.link)
                if e.published:
                    published_date = "Date: " + e.published.isoformat()[:10]
                else:
                    published_date = "Date: Unknown"
                if mark_as_read:
                    self._reader.mark_entry_as_read(e)

                feed_title = f"[bold]{self._get_feed_title(e.original_feed_url)}[/bold]"

                if e.important:
                    panel_body = ":exclamation_mark:"
                else:
                    panel_body = ""
                panel_body += f"Title: [bold]{e.title}[/bold]" + "\n"
                panel_body += str(published_date) + "\n\n"
                panel_body += strip_html(e.summary).strip() + "\n"

                console.print(
                    Panel(
                        panel_body,
                        title=feed_title,
                        subtitle=f"[link={e.link}]{e.link}[/link]",
                    )
                )
                console.print()
            if len(displayed_links) == limit:
                return

    def _get_feed_title(self, url: str):
        """Get display title for pynewsreader feed"""
        if url in self._feed_names and self._feed_names[url] is not None:
            return self._feed_names[url]
        elif self._reader.get_feed(url).title:
            return self._reader.get_feed(url).title
        else:
            return self._reader.get_feed(url).url

    def _get_entries(
        self, important: bool = None, read: Union[None, bool] = None, limit: int = 10
    ) -> List[reader.Entry]:
        """Get entries in reader.Entry format"""
        return self._reader.get_entries(read=read, limit=limit, important=important)

    def _get_tags(self, entry: reader.Entry):
        """Get tags for a given entry"""
        return [i[0] for i in list(self._reader.get_tags(entry))]

    def _mark_matching_entries_as_read(self, match_strings: List):
        for i in self._reader.get_entries(read=False):
            for filter_string in match_strings:
                if filter_string in i.title:
                    print(f"Marking entry as read: {i.title}")
                    self._reader.mark_entry_as_read(i)
                if filter_string in i.link:
                    print(f"Marking entry as read: {i.title}")
                    self._reader.mark_entry_as_read(i)

    def _mark_matching_entries_as_important(self, match_strings: List):
        for i in self._reader.get_entries(read=False):
            for filter_string in match_strings:
                if filter_string in i.title:
                    print(f"Marking entry as important: {i.title}")
                    self._reader.mark_entry_as_important(i)

    def _mark_important(self, entry: reader.Entry = None):
        """Mark entry as important"""
        if entry is not None:
            reader.Reader.mark_entry_as_important(entry)

    def _mark_unimportant(self, entry: reader.Entry = None):
        """Mark entry as important"""
        if entry is not None:
            reader.Reader.mark_entry_as_unimportant(entry)

    def _add_tag(self, entry: reader.Entry, tag_key: str, tag_value: Dict = None):
        """Add tag to entry"""
        reader.Reader.set_tag(entry, tag_key, tag_value)

    def _remove_tag(self, entry: reader.Entry, tag_key: str):
        """Remove tag from entry"""
        self._reader.delete_tag(entry, tag_key)

In [ ]:
# | export

@patch_to(PyNewsReader)
def blacklist_add(self, blacklist_string: str):
    """Add entry to blacklist"""
    if blacklist_string not in self._title_blacklist:
        self._title_blacklist.append(blacklist_string)
        with open(self._dbfolder / "title_blacklist.json", "wt") as myfile:
            json.dump(self._title_blacklist, myfile)


@patch_to(PyNewsReader)
def blacklist_remove(self, blacklist_string: str):
    """Remove entry from blacklist"""
    if blacklist_string in self._title_blacklist:
        self._title_blacklist.remove(blacklist_string)
        with open(self._dbfolder / "title_blacklist.json", "wt") as myfile:
            json.dump(self._title_blacklist, myfile)

@patch_to(PyNewsReader)
def blacklist_show(self):
    """Show blacklist"""
    print(self._title_blacklist)
  

In [ ]:
#| export

@patch_to(PyNewsReader)
def whitelist_add(self, whitelist_string: str):
    """Add entry to whitelist"""
    if whitelist_string not in self._title_whitelist:
        self._title_whitelist.append(whitelist_string)
        with open(self._dbfolder / "title_whitelist.json", "wt") as myfile:
            json.dump(self._title_whitelist, myfile)
        for entry in self._get_entries(limit=None):
            if whitelist_string in entry.title:
                self._reader.mark_entry_as_important(entry)


@patch_to(PyNewsReader)
def whitelist_remove(self, whitelist_string: str):
    """Remove whitelist entry"""
    if whitelist_string in self._title_whitelist:
        self._title_whitelist.remove(whitelist_string)
        with open(self._dbfolder / "title_whitelist.json", "wt") as myfile:
            json.dump(self._title_whitelist, myfile)
        for entry in self._get_entries(limit=None):
            if whitelist_string in entry.title:
                self._reader.mark_entry_as_unimportant(entry)
                
@patch_to(PyNewsReader)
def whitelist_show(self):
    """Show whitelist"""
    print(self._title_whitelist)
  

In [ ]:
# | export

@patch_to(PyNewsReader)
def update(self):
    """Update feeds and search"""
    self._reader.update_feeds()
    self._reader.update_search()
    if len(self._title_blacklist) > 0:
        self._mark_matching_entries_as_read(self._title_blacklist)
    if len(self._title_whitelist) > 0:
        self._mark_matching_entries_as_important(self._title_whitelist)

In [ ]:
# | export

@patch_to(PyNewsReader)
def add_feed(self, feed: Union[Feed, str]):
    """Add feed to pynewsreader

    Args:
        feed (Feed): pynewsreader Feed to add
    """
    if isinstance(feed, Feed):
        self._feed_names[feed.url] = feed.name
        self._reader.add_feed(feed.url.rstrip("/"), exist_ok=True)
    elif isinstance(feed, str):
        self._reader.add_feed(feed.rstrip("/"), exist_ok=True)
    else:
        raise Exception("Must be str or Feed type to add")

    # Save names to file
    with open(self._dbfolder / "feed_names.json", "wt") as myfile:
        json.dump(self._feed_names, myfile)


@patch_to(PyNewsReader)
def remove_feed(self, feed: Union[Feed, str]):
    """Remove feed from pynewsreader instance

    Args:
        feed (Union[Feed, str]): Feed to remove
    """

    if isinstance(feed, Feed):
        self._reader.delete_feed(feed.url)
    elif isinstance(feed, str):
        self._reader.delete_feed(feed)
    else:
        raise Exception(TypeError)


@patch_to(PyNewsReader)
def feeds(self):
    """List pynewsreader feeds

    Returns:
        List[str]: List of names of current pynewsreader feeds
    """
    return [self._get_feed_title(i.url) for i in self._reader.get_feeds()]

In [ ]:
# | export

@patch_to(PyNewsReader)
def show(
    self,
    limit: int = 6,
    read: bool = False,
    important: bool = None,
    mark_as_read: bool = True,
):
    """Pretty print entries

    Args:
        limit (int, optional): Number of entries to show. Defaults to 5.
        read (bool, optional): Show read entries (True), unread entries (False), or all entries (None). Defaults to None.
        mark_as_read (bool, optional): Mark displayed entries as read. Defaults to False.
    """
    self._print_entries(
        self._get_entries(read=read, important=important, limit=limit * 2),
        limit=limit,
        mark_as_read=mark_as_read,
    )

In [ ]:
# | export

@patch_to(PyNewsReader)
def search(self, query: str, mark_as_read: bool = True, limit: int = 10):
    """Search entries and pretty print results

    Args:a
        query (str): Search query
        mark_as_read (bool, optional): Mark results as read? Defaults to True.
    """
    self._print_entries(
        [self._reader.get_entry(i) for i in self._reader.search_entries(query)],
        mark_as_read=mark_as_read,
        limit=limit,
    )


In [ ]:
#| export

@patch_to(PyNewsReader)
def _mark_entries_important_by_url(self, url: str):
    matches = [i for i in self._reader.get_entries() if i.link==url]
    for article in matches:
        self._reader.mark_entry_as_important(article)


@patch_to(PyNewsReader)
def _mark_entries_unimportant_by_url(self, url: str):
    matches = [i for i in self._reader.get_entries() if i.link==url]
    for article in matches:
        self._reader.mark_entry_as_unimportant(article)


In [ ]:
r = PyNewsReader()

## Add Feeds

In [ ]:
# With names
for i in [
    Feed(url="https://ricochet.media/en/feed", name="Richochet Media"),
    Feed(url="https://thetyee.ca/rss2.xml", name="The Tyee"),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
        name="Toronto Star | Top Stories",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
        name="Toronto Star | Investigations",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
        name="Toronto Star | Editorials",
    ),
    Feed(url="https://www.macleans.ca/feed/", name="Macleans"),
]:
    r.add_feed(i)

# Use the default name from the feed
for i in [
    "https://rss.cbc.ca/lineup/topstories.xml",
    "https://rss.cbc.ca/lineup/world.xml",
    "https://rss.cbc.ca/lineup/canada.xml",
    "https://rss.cbc.ca/lineup/business.xml",
    "https://rss.cbc.ca/lineup/technology.xml",
    "https://www.cbc.ca/cmlink/rss-canada-ottawa",
    "https://thenarwhal.ca/feed/",
    "https://www.theguardian.com/science/rss",
    "https://www.theguardian.com/world/canada/rss",
]:
    r.add_feed(Feed(i))

## Update Feeds

In [ ]:
r.update()

Marking entry as read: Google launching its own artificial intelligence bot, known as Bard

Marking entry as read: Google launching its own artificial intelligence bot, known as Bard

Marking entry as read: 'Crypto King' associate wipes iPhone, hands over $1M in bank drafts payable to Aiden 
Pleterski

Marking entry as read: Elon Musk cleared in trial over tweets about taking Tesla private

Marking entry as read: ‘Tip of the iceberg’: why abuse in Canadian sport is worse than it seems

## List Feeds

In [ ]:
r.feeds()

['Canada | The Guardian',
 'CBC | Business News',
 'CBC | Canada News',
 'CBC | Ottawa News',
 'CBC | Technology News',
 'CBC | Top Stories News',
 'CBC | World News',
 'Macleans',
 'Richochet Media',
 'Science | The Guardian',
 'The Narwhal',
 'The Tyee',
 'Toronto Star | Investigations',
 'Toronto Star | Editorials',
 'Toronto Star | Top Stories']

## Display Entries

`read` = None shows all entries

`read` = False shows only unread entries

`read` = True shows only read entries

In [ ]:
r.show(limit=5, read=None, mark_as_read=False)

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: BC Pledges to Spend Surplus Billions to Improve Services (in News)                                       │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ Eby’s first throne speech says government’s priorities remain unchanged.                                        │
│                                                                                                                 │
╰─────────────── ]8;id=777259;https://thetyee.ca/News/2023/02/06/BC-Pledges-To-Spend-Surplus-Billions-Services/\https://thetyee.ca/News/2023/02/06/BC-Pledges-To-Spend-Surplus-Billions-Services/]8;;\ ───────────────╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: CONTEST: Win Tickets to Local Theatre for Black History Month (in Presents)                              │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ One lucky Tyee reader will get two tickets to ‘Broken Chord’ this February.                                     │
│                                                                                                                 │
╰───────────────────── ]8;id=557021;https://thetyee.ca/Presents/2023/02/06/Contest-Broken-Chord-Tickets/\https://thetyee.ca/Presents/2023/02/06/Contest-Broken-Chord-Tickets/]8;;\ ──────────────────────╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: Nicole Chan Deserved Safety Within Her VPD Workplace (in Opinion)                                        │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ Her tragic death tells us we must do more for survivors of institutional sexual abuse.                          │
│                                                                                                                 │
╰─────────────────── ]8;id=459322;https://thetyee.ca/Opinion/2023/02/06/Nicole-Chan-Deserved-Safety-At-VPD/\https://thetyee.ca/Opinion/2023/02/06/Nicole-Chan-Deserved-Safety-At-VPD/]8;;\ ───────────────────╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: Please Advise! Will the Smith-Trudeau Cage Match Be Worth Watching? (in Opinion)                         │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ Perhaps, says Dr. Steve. Anytime the PM and premiers meet on health care there are lots of potential fireworks. │
│                                                                                                                 │
╰──────────────────────── ]8;id=703269;https://thetyee.ca/Opinion/2023/02/06/Smith-Trudeau-Cage-Match/\https://thetyee.ca/Opinion/2023/02/06/Smith-Trudeau-Cage-Match/]8;;\ ────────────────────────╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: Why a Top BC Heart Surgeon Quit for Politics (in News)                                                   │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ Dr. Sanjiv Gandhi saved children’s lives in the operating room. Now he is a deputy leader of the BC Greens.     │
│                                                                                                                 │
╰──────────────────── ]8;id=296264;https://thetyee.ca/News/2023/02/06/BC-Heart-Surgeon-Quit-For-Politics/\https://thetyee.ca/News/2023/02/06/BC-Heart-Surgeon-Quit-For-Politics/]8;;\ ─────────────────────╯

## Search Entries

In [ ]:
r.search("rate", mark_as_read=False)

╭────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╮
│ ❗Title: Bank of Canada raises key interest rate for eighth time, to 4.5 per cent. What you need to know        │
│ Date: 2023-01-25                                                                                                │
│                                                                                                                 │
│ Central bank signals its rate-hiking campaign could finally be over, while it “assesses the impact of the       │
│ cumulative interest rate increases.”                                                                            │
│                                                                                                                 │
╰─ ]8;id=793898;https://www.thestar.com/business/2023/01/25/bank-of-canada-expected-to-hike-interest-rates-for-the-eighth-straight-time.html\https://www.thestar.com/business/2023/01/25/bank-of-canada-expected-to-hike-interest-rates-for-the-eighth-stra]8;;\─╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ ❗Title: Bank of Canada raises interest rate again — but hits 'pause' on any more for now                       │
│ Date: 2023-01-25                                                                                                │
│                                                                                                                 │
│ The Bank of Canada has raised its benchmark interest rate again, to 4.5 per cent, but hinted that it may now be │
│ done with its aggressive campaign of rate hikes.                                                                │
│                                                                                                                 │
╰──────────── ]8;id=204253;https://www.cbc.ca/news/business/bank-of-canada-january-rate-decision-1.6725283?cmp=rss\https://www.cbc.ca/news/business/bank-of-canada-january-rate-decision-1.6725283?cmp=rss]8;;\ ────────────╯

╭─────────────────────────────────────────────── CBC | World News ────────────────────────────────────────────────╮
│ Title: U.S. inflation rate was 6.5% in December                                                                 │
│ Date: 2023-01-12                                                                                                │
│                                                                                                                 │
│ The inflation rate in the U.S. decelerated to 6.5 per cent in December, mostly due to much cheaper gasoline.    │
│                                                                                                                 │
╰─────────────────── ]8;id=565848;https://www.cbc.ca/news/business/us-inflation-december-1.6711469?cmp=rss\https://www.cbc.ca/news/business/us-inflation-december-1.6711469?cmp=rss]8;;\ ────────────────────╯

╭─────────────────────────────────────────── Toronto Star | Editorials ───────────────────────────────────────────╮
│ ❗Title: With inflation easing will interest rates hikes pause?                                                 │
│ Date: 2023-01-20                                                                                                │
│                                                                                                                 │
│ With some welcome downward movement on inflation, let’s hope then that we’re seeing an end to the Bank of       │
│ Canada’s string of rate hikes.                                                                                  │
│                                                                                                                 │
╰─ ]8;id=679913;https://www.thestar.com/opinion/editorials/2023/01/20/with-inflation-easing-will-interest-rates-hikes-pause.html\https://www.thestar.com/opinion/editorials/2023/01/20/with-inflation-easing-will-interest-rates-hikes-pause.ht]8;;\─╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: Inflation rate slows to 6.3% — but groceries are still going up at nearly twice that pace                │
│ Date: 2023-01-17                                                                                                │
│                                                                                                                 │
│ Canada's annual inflation rate cooled to 6.3 per cent in December, its lowest level since February.             │
│                                                                                                                 │
╰───────────────── ]8;id=796431;https://www.cbc.ca/news/business/canada-inflation-december-1.6716315?cmp=rss\https://www.cbc.ca/news/business/canada-inflation-december-1.6716315?cmp=rss]8;;\ ──────────────────╯

╭────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╮
│ Title: Vacancy rate for downtown Toronto offices climbs to 13.6 per cent                                        │
│ Date: 2023-01-12                                                                                                │
│                                                                                                                 │
│ This week, online retail giant Shopify put its massive new office space at The Well onto the sublet market.     │
│                                                                                                                 │
╰─ ]8;id=330399;https://www.thestar.com/business/2023/01/12/vacancy-rate-for-downtown-toronto-offices-climbs-to-136-per-cent.html\https://www.thestar.com/business/2023/01/12/vacancy-rate-for-downtown-toronto-offices-climbs-to-136-per-cent.h]8;;\─╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: More U.S. rate hikes in store as Federal Reserve warns inflation could 'spring back'                     │
│ Date: 2023-02-02                                                                                                │
│                                                                                                                 │
│ It's premature to declare victory over inflation, says the U.S. central bank's Jerome Powell, in a move that    │
│ could force North American interest rates even higher yet.                                                      │
│                                                                                                                 │
╰─────────────── ]8;id=658627;https://www.cbc.ca/news/business/fed-outlook-column-don-pittis-1.6733151?cmp=rss\https://www.cbc.ca/news/business/fed-outlook-column-don-pittis-1.6733151?cmp=rss]8;;\ ────────────────╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: U.S. added 517,000 jobs last month, pushing unemployment rate to lowest since 1969                       │
│ Date: 2023-02-03                                                                                                │
│                                                                                                                 │
│ The U.S. economy added more than half a million jobs in January, an astonishing number that pushed the jobless  │
│ rate down to its lowest level in half a century.                                                                │
│                                                                                                                 │
╰────────────────────── ]8;id=119676;https://www.cbc.ca/news/business/us-jobs-january-1.6735707?cmp=rss\https://www.cbc.ca/news/business/us-jobs-january-1.6735707?cmp=rss]8;;\ ───────────────────────╯

╭────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╮
│ ❗Title: Hundreds of thousands of jobs at risk if Bank of Canada raises key rate for eighth time Wednesday,     │
│ economists warn                                                                                                 │
│ Date: 2023-01-24                                                                                                │
│                                                                                                                 │
│ The economy still hasn’t felt the full impact of the previous seven hikes, and is already struggling, according │
│ to experts.                                                                                                     │
│                                                                                                                 │
╰─ ]8;id=938627;https://www.thestar.com/business/2023/01/24/hundreds-of-thousands-of-jobs-at-risk-if-bank-of-canada-raises-key-rate-for-eighth-time-wednesday-economists-warn.html\https://www.thestar.com/business/2023/01/24/hundreds-of-thousands-of-jobs-at-risk-if-bank-of-canada-raises-key]8;;\─╯

╭────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╮
│ Title: Justin Trudeau won’t discuss his approval ratings — but some Liberals are wondering if his message needs │
│ to change                                                                                                       │
│ Date: 2023-01-25                                                                                                │
│                                                                                                                 │
│ The prime minister says demonstrators don’t represent most Canadians, nor do they represent a broader shift in  │
│ public mood against him personally.                                                                             │
│                                                                                                                 │
╰─ ]8;id=844609;https://www.thestar.com/politics/federal/2023/01/25/justin-trudeau-wont-discuss-his-approval-ratings-but-some-liberals-are-wondering-if-his-message-needs-to-change.html\https://www.thestar.com/politics/federal/2023/01/25/justin-trudeau-wont-discuss-his-approval-ratings-but-some-]8;;\─╯

## Tag Entry

In [ ]:
test = list(r._reader.get_entries())

In [ ]:
r._reader.set_tag(test[0], "foobar")

In [ ]:
list(r._reader.get_tags(test[0]))

[('foobar', None)]

In [ ]:
# We added a method to just return the tag key:
r._get_tags(test[0])

['foobar']

## Remove tag from entry

In [ ]:
r._reader.delete_tag(test[0], "foobar")

In [ ]:
list(r._get_tags(test[0]))

[]

## Mark as Important/Unimportant

In [ ]:
r._reader.mark_entry_as_important(test[0])
r._reader.mark_entry_as_unimportant(test[0])

## Mark as Read / Unread

In [ ]:
r._reader.mark_entry_as_read(test[0])
r._reader.mark_entry_as_unread(test[0])

## Automatically mark entries as read

When updating feeds, entries with titles matching these strings will automatically be marked as read

In [ ]:
for i in [
    "Musk",
    "Apple",
    "Bezos",
    "Google",
    "Samsung",
    "iPhone",
    "iPad",
    "guardian.com/sport/",
    "guardian.com/football/",
    "thestar.com/sports",
]:
    r.blacklist_add(i)

In [ ]:
r.blacklist_show()

[
    'Musk',
    'Apple',
    'Bezos',
    'Google',
    'Samsung',
    'iPhone',
    'iPad',
    'guardian.com/sport/',
    'guardian.com/football/',
    'thestar.com/sports'
]

In [ ]:
# r.update()

## Automatically mark entries as important

In [ ]:
for i in ["interest rate", "Bank of Canada", "housing market"]:
    r.whitelist_add(i)

In [ ]:
r.whitelist_show()

['interest rate', 'Bank of Canada', 'housing market']

In [ ]:
# r.update()

## Low level access to data

In [ ]:
entries = r._get_entries(limit=1)
entry = [i for i in entries][0]
entry

Entry(id='https://thetyee.ca/News/2023/02/06/BC-Pledges-To-Spend-Surplus-Billions-Services/', updated=None, title='BC Pledges to Spend Surplus Billions to Improve Services (in News)', link='https://thetyee.ca/News/2023/02/06/BC-Pledges-To-Spend-Surplus-Billions-Services/', author='Andrew MacLeod', published=None, summary='Eby’s first throne speech says government’s priorities remain unchanged.', content=(), enclosures=(Enclosure(href='https://thetyee.ca/News/2023/02/06/DavidEbyWalking.jpg', type='image/jpeg', length=None),), read=False, read_modified=datetime.datetime(2023, 2, 7, 1, 4, 35, 113527, tzinfo=datetime.timezone.utc), important=False, important_modified=datetime.datetime(2023, 2, 7, 1, 4, 35, 104646, tzinfo=datetime.timezone.utc), added=datetime.datetime(2023, 2, 7, 1, 4, 26, 541098, tzinfo=datetime.timezone.utc), added_by='feed', last_updated=datetime.datetime(2023, 2, 7, 1, 4, 26, 541098, tzinfo=datetime.timezone.utc), original_feed_url='https://thetyee.ca/rss2.xml', feed=F

## CLI

In [ ]:
# | export


def main():
    fire.Fire(PyNewsReader)

In [ ]:
# | export


if __name__ == "__main__":
    main()

NAME
    ipykernel_launcher.py -f /home/mark/.local/share/jupyter/runtime/kernel-8f2da236-3f18-4738-ac02-9dc3b1087781.json

SYNOPSIS
    ipykernel_launcher.py -f /home/mark/.local/share/jupyter/runtime/kernel-8f2da236-3f18-4738-ac02-9dc3b1087781.json COMMAND

COMMANDS
    COMMAND is one of the following:

     add_feed
       Add feed to pynewsreader

     blacklist_add
       Add entry to blacklist

     blacklist_remove
       Remove entry from blacklist

     blacklist_show
       Show blacklist

     feeds
       List pynewsreader feeds

     remove_feed
       Remove feed from pynewsreader instance

     search
       Search entries and pretty print results

     show
       Pretty print entries

     update
       Update feeds and search

     whitelist_add
       Add entry to whitelist

     whitelist_remove
       Remove whitelist entry

     whitelist_show
       Show whitelist


In [ ]:
# | hide

# To Do:
# * User interface

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()